In [15]:
import json
import numpy as np

# define path for output file
output_path = './structure.json'
# dimensions of the tile (for positioning)
tx = 0.025
ty = 0.0135
tz = 0.034
# initialize output dictionary
output = {
    'suits': [],
    'tiles': []
}
# fill array of suit indices
four_ranges = [range(0, 9), range(10, 19), range(20, 29), range(30, 37)]
for fr in four_ranges:
    for i in fr:
        output['suits'] = output['suits'] + [i, i, i, i]
for i in range(40, 48):
    output['suits'] = output['suits'] + [i]
# fill array of tiles
pyramid = []
idx = 0
for l in range(0, 4):
    level = []
    for i in range(0, 8 - 2 * l):
        row = []
        for j in range(0, 8 - 2 * l):
            current_tile = {
                'tileIdx': idx,
                'position': (round((-3.5 + j + l)*tx, 5), l*ty, round((-3.5 + i + l)*tz, 5))
            }
            idx += 1
            # set right element
            if j == 8 - 2 * l - 1:
                current_tile['right'] = []
            else:
                current_tile['right'] = [idx]  # at this point index is updated for next tile
            # set under element
            if l > 0:
                current_tile['under'] = [
                    current_tile['tileIdx'] - (8 - 2 * (l - 1)) * (8 - 2 * (l - 1)) + 9 - 2 * (l - 1) + 2 * i]
            else:
                current_tile['under'] = []
            pyramid.append(current_tile)
# add tile at the top of the pyramid
pyramid.append({
    'tileIdx': idx,
    'right': [],
    'under': [116, 117, 118, 119],
    'position': (0.0, 4*ty, 0.0)
})
# add missing tiles
# will fix right tiles later
for i in range(121,144):
    pyramid.append({
        'tileIdx': i,
        'right': [],
        'under': [],
        'position': (0.0, 0.0, 0.0)
    })
# tiles are already sorted by index
pyramid[121]['right'] = [122]
pyramid[122]['right'] = [0]
pyramid[7]['right'] = [123]
pyramid[123]['right'] = [124]
pyramid[125]['right'] = [126]
pyramid[126]['right'] = [56]
pyramid[63]['right'] = [127]
pyramid[127]['right'] = [128]
pyramid[129]['right'] = [130,131]
pyramid[130]['right'] = [133]
pyramid[131]['right'] = [134]
pyramid[132]['right'] = [16]
pyramid[133]['right'] = [24]
pyramid[134]['right'] = [32]
pyramid[135]['right'] = [40]
pyramid[23]['right'] = [136]
pyramid[31]['right'] = [137]
pyramid[39]['right'] = [138]
pyramid[47]['right'] = [139]
pyramid[137]['right'] = [140]
pyramid[138]['right'] = [141]
pyramid[140]['right'] = [142]
pyramid[141]['right'] = [142]
pyramid[142]['right'] = [143]
# initialize left and over for all tiles
for tile in pyramid:
    tile['left'] = []
    tile['over'] = []
# fill in left and over for all tiles
for tile in pyramid:
    for idx in tile['right']:
        pyramid[idx]['left'].append(tile['tileIdx'])
    for idx in tile['under']:
        pyramid[idx]['over'].append(tile['tileIdx'])
# set position for last tiles (121-143)
while 0.0 in [t['position'][0] for t in pyramid if t['tileIdx']>120]:
    for t in [tile for tile in pyramid if tile['tileIdx']>120]:
        if t['position'][0] == 0.0:
            for left in t['left']:
                if pyramid[left]['position'][0] != 0.0:
                    t['position'] = (round(pyramid[left]['position'][0] + tx, 5), t['position'][1],
                                     np.mean([pyramid[l]['position'][2] for l in t['left']]))
                    break
            for right in t['right']:
                if pyramid[right]['position'][0] != 0.0:
                    t['position'] = (round(pyramid[right]['position'][0] - tx, 5), t['position'][1],
                                     np.mean([pyramid[r]['position'][2] for r in t['right']]))
                    break
# associate list of tiles to output dictionary (re-order keys)
output['tiles'] = [{
    'tileIdx':tile['tileIdx'],
    'left':tile['left'],
    'right':tile['right'],
    'over':tile['over'],
    'under':tile['under'],
    'position':tile['position'],
} for tile in pyramid]
# export dictionary
with open(output_path, 'w') as file:
    json.dump(output, file, indent=4)